In [1]:
import pandas as pd

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df

,Lugar de residencia,Medida,Valor
0,Huelva,Población,510743
1,Huelva,Confirmados PDIA,2756
2,Huelva,Confirmados PDIA 14 días,947
3,Huelva,Tasa PDIA 14 días,"185,41614863052456"
4,Huelva,Confirmados PDIA 7 días,588
...,...,...,...
627,Municipio de Huelva sin especificar,Tasa PDIA 14 días,-
628,Municipio de Huelva sin especificar,Confirmados PDIA 7 días,15
629,Municipio de Huelva sin especificar,Total Confirmados,46
630,Municipio de Huelva sin especificar,Curados,8


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  2756.0


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  700.0


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 1168 personas en los últimos 7 días 

Un positivo PCR cada 577 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,2756.0,588.0,947.0,510743.0,115.126394,185.416149,307.0
Huelva-Costa,1399.0,277.0,485.0,288115.0,96.142165,168.335560,127.0
Sierra de Huelva-Andévalo Central,436.0,158.0,198.0,67571.0,233.828122,293.025114,98.0
Condado-Campiña,875.0,138.0,246.0,155057.0,88.999529,158.651335,70.0
Huelva (capital),700.0,123.0,249.0,143663.0,85.617034,173.322289,57.0
Cortegana,91.0,68.0,79.0,4666.0,1457.351050,1693.099014,42.0
Valverde del Camino,59.0,25.0,29.0,12820.0,195.007800,226.209048,26.0
Lepe,211.0,48.0,78.0,27431.0,174.984507,284.349823,20.0
Almonte,113.0,36.0,53.0,24191.0,148.815675,219.089744,18.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() > 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Granado (El),31.0,13.0,29.0,516.0,2519.379845,5620.155039,5.0
Cortegana,91.0,68.0,79.0,4666.0,1457.351050,1693.099014,42.0
Villarrasa,34.0,5.0,18.0,2176.0,229.779412,827.205882,3.0
Cerro de Andévalo (El),22.0,11.0,17.0,2364.0,465.313029,719.120135,1.0
Puerto Moral,2.0,2.0,2.0,282.0,709.219858,709.219858,1.0
Almonaster la Real,12.0,10.0,11.0,1819.0,549.752611,604.727872,6.0
Marines (Los),2.0,2.0,2.0,396.0,505.050505,505.050505,0.0
Palma del Condado (La),135.0,26.0,47.0,10761.0,241.613233,436.762383,17.0
Valdelarco,3.0,1.0,1.0,239.0,418.410042,418.410042,0.0
